## Index
Click on these links for each section that you need to correct. Thanks for correcting my notebook!

1. [Part 1](#section 1)
2. [Part 2](#section 2)
3. [Part 3](#section 3)

<a id='section 1'></a>
<h2><u> Part 1: Finding the shape of the pre-processed table </u></h2>

In [2]:
import requests
from scrapy.http import TextResponse
from bs4 import BeautifulSoup
import re 
import pandas as pd

<h3> Functions used to parse a website, remove html tags and remove special chars like \n</h3>

In [38]:
#parses web link and returns matches found for given tag
def parser(link, tag):
    res = requests.get(link)
    response = TextResponse(res.url, body=res.text, encoding='utf-8')
    return response.css(tag).getall()
    
#function to remove html tags
def remove_html(movies):
    sol = []
    for movie in movies:
        s = BeautifulSoup(movie, "lxml").text
        if re.search('[a-zA-Z]', s):
            sol.append(s)
    return sol

<h3> Parse the website, remove html tags and special chars </h3>

In [41]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tag = 'td'
table = parser(link, tag)
#removed the data outside of the table and remove the html tags scraped 
table = remove_html(table[:-34])
table

['M1A',
 'Not assigned',
 'Not assigned\n',
 'M2A',
 'Not assigned',
 'Not assigned\n',
 'M3A',
 'North York',
 'Parkwoods\n',
 'M4A',
 'North York',
 'Victoria Village\n',
 'M5A',
 'Downtown Toronto',
 'Harbourfront\n',
 'M6A',
 'North York',
 'Lawrence Heights\n',
 'M6A',
 'North York',
 'Lawrence Manor\n',
 'M7A',
 'Downtown Toronto',
 "Queen's Park\n",
 'M8A',
 'Not assigned',
 'Not assigned\n',
 'M9A',
 "Queen's Park",
 'Not assigned\n',
 'M1B',
 'Scarborough',
 'Rouge\n',
 'M1B',
 'Scarborough',
 'Malvern\n',
 'M2B',
 'Not assigned',
 'Not assigned\n',
 'M3B',
 'North York',
 'Don Mills North\n',
 'M4B',
 'East York',
 'Woodbine Gardens\n',
 'M4B',
 'East York',
 'Parkview Hill\n',
 'M5B',
 'Downtown Toronto',
 'Ryerson\n',
 'M5B',
 'Downtown Toronto',
 'Garden District\n',
 'M6B',
 'North York',
 'Glencairn\n',
 'M7B',
 'Not assigned',
 'Not assigned\n',
 'M8B',
 'Not assigned',
 'Not assigned\n',
 'M9B',
 'Etobicoke',
 'Cloverdale\n',
 'M9B',
 'Etobicoke',
 'Islington\n',
 'M9B

<h3> Create the pandas dataframe by first grouping every 3 values together in a nested list and then converting it into a pandas dataframe </h3>

In [42]:
#grouping every 3 values a single list inside a list
#easier to make this list and to then append it into a pandas dataframe!
i = 3
lis = []
while i <= len(table):
    lis.append(table[i-3:i])
    i += 3
lis

[['M1A', 'Not assigned', 'Not assigned\n'],
 ['M2A', 'Not assigned', 'Not assigned\n'],
 ['M3A', 'North York', 'Parkwoods\n'],
 ['M4A', 'North York', 'Victoria Village\n'],
 ['M5A', 'Downtown Toronto', 'Harbourfront\n'],
 ['M6A', 'North York', 'Lawrence Heights\n'],
 ['M6A', 'North York', 'Lawrence Manor\n'],
 ['M7A', 'Downtown Toronto', "Queen's Park\n"],
 ['M8A', 'Not assigned', 'Not assigned\n'],
 ['M9A', "Queen's Park", 'Not assigned\n'],
 ['M1B', 'Scarborough', 'Rouge\n'],
 ['M1B', 'Scarborough', 'Malvern\n'],
 ['M2B', 'Not assigned', 'Not assigned\n'],
 ['M3B', 'North York', 'Don Mills North\n'],
 ['M4B', 'East York', 'Woodbine Gardens\n'],
 ['M4B', 'East York', 'Parkview Hill\n'],
 ['M5B', 'Downtown Toronto', 'Ryerson\n'],
 ['M5B', 'Downtown Toronto', 'Garden District\n'],
 ['M6B', 'North York', 'Glencairn\n'],
 ['M7B', 'Not assigned', 'Not assigned\n'],
 ['M8B', 'Not assigned', 'Not assigned\n'],
 ['M9B', 'Etobicoke', 'Cloverdale\n'],
 ['M9B', 'Etobicoke', 'Islington\n'],
 ['M9

In [55]:
#every 3 values needs to be inserted into a pandas dataframe to make the final table
df = pd.DataFrame(lis, columns = ['PostalCode', 'Borough', 'Neighborhood'])
df = df.replace('\n',' ', regex=True)
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


<h3> Here I drop the not assigned boroughs and reset the index </h3>

In [56]:
#drop cells with not assigned borough
df = df[df.Borough != 'Not assigned']
df.reset_index(drop = True, inplace = True)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


<h3> This cell groups by Postal Code and Borough (since some codes have multiple boroughs) and then joins the Neighborhoods together using a comma. I then reset the index to make the PostalCode repeat according to how we want the dataframe to look.</h3>

In [61]:
g = df.reset_index().groupby(['PostalCode', 'Borough'], as_index = False)['Neighborhood'].apply(', '.join)
g = g.to_frame().reset_index()
g.columns = ['PostalCode', 'Borough', 'Neighborhood']
g

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park"
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge"
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W..."
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


<h3>The code below finds the cell with not assigned neighborhood and assign it to that cells Borough value</h3>

In [62]:
g.loc[g.Neighborhood == 'Not assigned', 'Neighborhood'] = g.loc[g.Neighborhood == 'Not assigned', 'Borough']
print("There are", len(g.loc[g.Neighborhood == 'Not assigned']), "'not assigned neighborhood' cells")

There are 0 'not assigned neighborhood' cells


<h3> Final shape answer </h3>

In [63]:
final_df = g
print("The shape of the fully pre-processed dataframe is: ", final_df.shape)

The shape of the fully pre-processed dataframe is:  (103, 3)


<a id='section 2'></a>
<h2><u> Part 2: Finding latitude and longitude </u></h2>

In [66]:
codes = pd.read_csv('g_co.csv')
codes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [67]:
df_lat_long = final_df.join(codes.set_index('Postal Code'), on='PostalCode')

In [68]:
df_lat_long

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W...",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


<a id='section 3'></a>
<h2><u> Part 3: Clustering of Toronto neighborhoods </u></h2>

In [71]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<h3> Filter for Boroughs of Toronto </h3>

In [83]:
toronto = df_lat_long[df_lat_long['Borough'].str.find("Toronto") != -1]
toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West , India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park , Forest Hill SE , Rathnelly , South...",43.686412,-79.400049


<h3> Create folium map of Toronto neighborhoods </h3>

In [79]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [85]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [86]:
CLIENT_ID = 'NPMMARBIRQR5D10GGDYDGDR20UWYFGLYMGYUILWNMHIWI5HI' # your Foursquare ID
CLIENT_SECRET = 'NSDJFWX3A4LUWQP5BTWW3Q4XTDETWUCNXM1UWQAYNOCSE4NR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NPMMARBIRQR5D10GGDYDGDR20UWYFGLYMGYUILWNMHIWI5HI
CLIENT_SECRET:NSDJFWX3A4LUWQP5BTWW3Q4XTDETWUCNXM1UWQAYNOCSE4NR


In [89]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )



The Beaches 
The Danforth West , Riverdale 
The Beaches West , India Bazaar 
Studio District 
Lawrence Park 
Davisville North 
North Toronto West 
Davisville 
Moore Park , Summerhill East 
Deer Park , Forest Hill SE , Rathnelly , South Hill , Summerhill West 
Rosedale 
Cabbagetown , St. James Town 
Church and Wellesley 
Harbourfront 
Ryerson , Garden District 
St. James Town 
Berczy Park 
Central Bay Street 
Adelaide , King , Richmond 
Harbourfront East , Toronto Islands , Union Station 
Design Exchange , Toronto Dominion Centre 
Commerce Court , Victoria Hotel 
Roselawn 
Forest Hill North , Forest Hill West 
The Annex , North Midtown , Yorkville 
Harbord , University of Toronto 
Chinatown , Grange Park , Kensington Market 
CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara 
Stn A PO Boxes 25 The Esplanade 
First Canadian Place , Underground city 
Christie 
Dovercourt Village , Dufferin 
Little Portugal , Trinity 
Brockton ,

In [90]:
print(toronto_venues.shape)
toronto_venues.head()

(1698, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West , Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [91]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide , King , Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton , Exhibition Place , Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara",14,14,14,14,14,14
"Cabbagetown , St. James Town",43,43,43,43,43,43
Central Bay Street,82,82,82,82,82,82
"Chinatown , Grange Park , Kensington Market",86,86,86,86,86,86
Christie,17,17,17,17,17,17


In [92]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 225 uniques categories.


### Analyze Each Neighborhood

In [94]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,T

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [95]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide , King , Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.0000,0.000000,0.000000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.00,0.000000,0.040000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.070000,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.030000,0.00000,0.000000,0.000000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

### Top 5 most common venues

In [97]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide , King , Richmond ----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.04
2       Steakhouse  0.04
3   Cosmetics Shop  0.03
4  Thai Restaurant  0.03


----Berczy Park ----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2        Beer Bar  0.04
3  Farmers Market  0.04
4            Café  0.04


----Brockton , Exhibition Place , Parkdale Village ----
            venue  freq
0     Coffee Shop  0.09
1            Café  0.09
2  Breakfast Spot  0.09
3          Bakery  0.05
4       Nightclub  0.05


----Business Reply Mail Processing Centre 969 Eastern ----
                  venue  freq
0           Yoga Studio  0.06
1         Auto Workshop  0.06
2            Comic Shop  0.06
3                  Park  0.06
4  Gym / Fitness Center  0.06


----CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara ----
              venue  freq
0   Airport Service  0.21
1    Airport Lounge  0.14
2 

In [100]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [137]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide , King , Richmond",Coffee Shop,Café,Steakhouse,Restaurant,Asian Restaurant,Bakery,Bar,Cosmetics Shop,Thai Restaurant,Sushi Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Café,Beer Bar,Bakery,Seafood Restaurant,Cheese Shop,Steakhouse,Shopping Mall
2,"Brockton , Exhibition Place , Parkdale Village",Coffee Shop,Breakfast Spot,Café,Climbing Gym,Stadium,Bar,Bakery,Restaurant,Intersection,Pet Store
3,Business Reply Mail Processing Centre 969 East...,Yoga Studio,Park,Restaurant,Fast Food Restaurant,Farmers Market,Auto Workshop,Spa,Pizza Place,Burrito Place,Light Rail Station
4,"CN Tower , Bathurst Quay , Island airport , Ha...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Boutique,Airport,Airport Food Court,Sculpture Garden,Bar,Harbor / Marina


### Run k-means clustering on the grouped data

In [138]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [139]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Trail,Health Food Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
41,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Diner,Sports Bar,Liquor Store,Dessert Shop
42,M4L,East Toronto,"The Beaches West , India Bazaar",43.668999,-79.315572,0,Sandwich Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Pub,Sushi Restaurant,Pizza Place,Food & Drink Shop,Ice Cream Shop,Pet Store
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Lawyer,Bus Line,Swim School,Women's Store,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [140]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 0: Pub Cluster

In [141]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Pub,Trail,Health Food Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Diner,Sports Bar,Liquor Store,Dessert Shop
42,East Toronto,0,Sandwich Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Pub,Sushi Restaurant,Pizza Place,Food & Drink Shop,Ice Cream Shop,Pet Store
43,East Toronto,0,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop
45,Central Toronto,0,Department Store,Park,Breakfast Spot,Gym,Sandwich Place,Food & Drink Shop,Hotel,Women's Store,Dumpling Restaurant,Donut Shop
46,Central Toronto,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park,Chinese Restaurant,Café
47,Central Toronto,0,Sandwich Place,Pizza Place,Dessert Shop,Coffee Shop,Gym,Café,Sushi Restaurant,Italian Restaurant,Gas Station,Indian Restaurant
49,Central Toronto,0,Coffee Shop,Pub,Sushi Restaurant,Fried Chicken Joint,Sports Bar,Supermarket,Liquor Store,Pizza Place,Athletics & Sports,Light Rail Station
51,Downtown Toronto,0,Coffee Shop,Pizza Place,Restaurant,Italian Restaurant,Pub,Bakery,Café,Ice Cream Shop,Playground,Park
52,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Gay Bar,Japanese Restaurant,Restaurant,Gym,Hotel,Café,Burger Joint,Men's Store


### Cluster 1-4: Playground/Park/Garden Clusters

In [142]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Playground,Tennis Court,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [143]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,2,Park,Trail,Playground,Women's Store,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Garden,Home Service,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [145]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4,Park,Lawyer,Bus Line,Swim School,Women's Store,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
